In [1]:
import pandas as pd
import numpy as np

%matplotlib inline
%load_ext autoreload
%autoreload 2

In [2]:
data_path = './data/'
!ls -lthr data/

total 5766736
-r--r--r--@ 1 samshi  staff   507M 20 Feb 20:18 test.csv
-r--r--r--@ 1 samshi  staff   1.9G 20 Feb 20:18 train.csv
-r--r--r--@ 1 samshi  staff   246M 20 Feb 20:19 item_metadata.csv
-r--r--r--@ 1 samshi  staff    51M 20 Feb 20:25 submission_popular.csv


In [3]:
%%time
# nrows = 10000
nrows = None
train = pd.read_csv(data_path+'train.csv', nrows=nrows)

In [25]:
test = pd.read_csv(data_path+'test.csv', nrows=nrows)
test.shape

(3782335, 12)

In [7]:
train.shape

(15932992, 12)

In [4]:
train.head()

,user_id,session_id,timestamp,step,action_type,reference,platform,city,device,current_filters,impressions,prices
0,008UN2RY0DMY,cd10940538005,1541031243,1,search for destination,"Rochester, USA",US,"Rochester, USA",desktop,NaN,NaN,NaN
1,008UN2RY0DMY,cd10940538005,1541031463,2,change of sort order,interaction sort button,US,"Rochester, USA",desktop,NaN,NaN,NaN
2,008UN2RY0DMY,cd10940538005,1541031473,3,change of sort order,distance and recommended,US,"Rochester, USA",desktop,NaN,NaN,NaN
3,008UN2RY0DMY,cd10940538005,1541031473,4,filter selection,Focus on Distance,US,"Rochester, USA",desktop,Focus on Distance,NaN,NaN
4,008UN2RY0DMY,cd10940538005,1541031550,5,interaction item image,57103,US,"Rochester, USA",desktop,NaN,NaN,NaN


In [6]:
train.action_type.unique()

array(['search for destination', 'change of sort order',
       'filter selection', 'interaction item image',
       'interaction item info', 'interaction item deals', 'clickout item',
       'search for item', 'search for poi'], dtype=object)

### some case reference is not in the impressions
we may keep or discard these, it might help as maybe the artifacts have pattern not otherwise

In [8]:
click_mask = ~train.impressions.isna()

In [9]:
not_ins = train[click_mask].apply(lambda row: row.reference not in row.impressions.split('|'), axis=1)

In [10]:
sum(not_ins)

749

In [11]:
# check if all sessions come with a click-out
trn_grp = train.groupby('session_id')

In [12]:
def check_click(grp):
    return 'clickout item' in grp.action_type.unique()

has = trn_grp.apply(check_click)

uids_has = train[train.session_id.isin(has[has].index)].user_id.unique()
uids_not_has = train[train.session_id.isin(has[~has].index)].user_id.unique()

In [16]:
len(uids_not_has)

68092

In [17]:
len(uids_not_has)/len(train)

0.004273648037983073

In [19]:
len(set(uids_not_has).intersection(set(uids_has)))

55063

In [21]:
set(uids_not_has) - set(uids_has)

{'5O1A5UL3LDTG',
 'KTEB4AYPL5OJ',
 'XP24MP03NMNK',
 '88H4C3MIF39R',
 'A5N3UAITP3JH',
 'FH55JU4MDFIE',
 'C4JW0AA8AHPZ',
 '63K50ANI486L',
 'LQ7R6TURTOCD',
 '6UINOIN0SWFM',
 'M7M3LC59GMHP',
 'Q9M9PLM1ER2D',
 'D7BLYH27KZ9W',
 'J27GVHRSNL4W',
 'TMRCPBMPJC7V',
 'XQBKGXR267LI',
 'Y4QKUHF9RW3C',
 'IEPJPR55RC8G',
 '3F4BFS41AZFP',
 'R8LD58GKCC1V',
 '2IVXY4O882AB',
 'R5WHFQSAW158',
 '3ZROT0YARU3U',
 'T5KP4AR49ZL8',
 'T5SR1AIEQ8M1',
 'C0A97DU33XU5',
 '3L3P46JJEQQJ',
 '1D50KM7KB832',
 '3W4MJ9C5UVA2',
 '07DT5YJ8SQMN',
 '1ZSWNUBVFXWH',
 '2FP99URI8K1Y',
 'DPP38MAHFYAN',
 'FZQGR5XLRYO0',
 '3WFOMSQ8FFEM',
 '0FIX1Q11Q4X8',
 'Y9RDO8X7NT17',
 'DP6VLOO0QUVM',
 '8YBJ8LM79131',
 'KCLATU22OSSD',
 '8RENJ4NKJKAQ',
 'WLYSMVLLE5S2',
 '1ZWV6E70XNP3',
 'PAZMY8Y8L497',
 'PVDVBTU941OG',
 'K53ZE7HDAFNH',
 'SYFTC0ZH7195',
 'P877C2P3NI62',
 'V6ZINZN98158',
 'F40RTBVBRGK8',
 '52V3E2WS20GE',
 'ZB269TF5CTV3',
 'K5556DGKSPEM',
 'IMUZ7KVCQ990',
 'RD51NF857TY4',
 'XY4DPT3Q6PSA',
 '2HGDDFL26UUM',
 'I1M88NXTWHU5',
 'Y7UN9QC2TP2K

In [24]:
train[train.user_id=='5O1A5UL3LDTG'].action_type.unique()

array(['interaction item image', 'change of sort order',
       'filter selection'], dtype=object)

In [14]:
train[train.user_id=='008UN2RY0DMY'].action_type.unique()

array(['search for destination', 'change of sort order',
       'filter selection', 'interaction item image',
       'interaction item info', 'clickout item', 'interaction item deals'],
      dtype=object)

In [15]:
train[train.session_id=='cd10940538005'].action_type.unique()

array(['search for destination', 'change of sort order',
       'filter selection', 'interaction item image',
       'interaction item info'], dtype=object)

In [27]:
len(set(test.user_id.unique()) - (set(uids_not_has) - set(uids_has)))

244151

In [31]:
len((set(uids_not_has) - set(uids_has)) - set(test[~test.impressions.isna()].user_id.unique()))

6328

In [32]:
len(set(uids_not_has) - set(uids_has))

13029

In [34]:
(set(uids_not_has) - set(uids_has)) - set(test[~test.impressions.isna()].user_id.unique())

{'KTEB4AYPL5OJ',
 'XP24MP03NMNK',
 'FH55JU4MDFIE',
 '6UINOIN0SWFM',
 'J27GVHRSNL4W',
 'XQBKGXR267LI',
 'Y4QKUHF9RW3C',
 'R8LD58GKCC1V',
 '3ZROT0YARU3U',
 '1D50KM7KB832',
 '3W4MJ9C5UVA2',
 '07DT5YJ8SQMN',
 'DPP38MAHFYAN',
 'FZQGR5XLRYO0',
 'Y9RDO8X7NT17',
 'DP6VLOO0QUVM',
 'WLYSMVLLE5S2',
 '8RENJ4NKJKAQ',
 'PVDVBTU941OG',
 'SYFTC0ZH7195',
 '52V3E2WS20GE',
 'ZB269TF5CTV3',
 'K5556DGKSPEM',
 'RD51NF857TY4',
 'XY4DPT3Q6PSA',
 '2HGDDFL26UUM',
 '4HM3SDBGQ434',
 'WV4QK37M6EPC',
 'LDQ21H6B6D72',
 'AJIUVG2BQCYB',
 'JBDXSW5MAK0I',
 'BU4K95465I2T',
 '18NF9T368345',
 'ZF35WXW10B0E',
 '4Q4K6DY8X6W7',
 '1V6XFKB8TTPW',
 'F9NZCQ6WN97R',
 'YN0MD6WOPCGQ',
 'C0BE1KM0C34I',
 'IXS7CAIXC5WA',
 '3MQOF8MMN308',
 'MLVJEPK37TSU',
 '3M089IKH20UJ',
 '1VS0I8I91349',
 'SH4PFS33YVGT',
 'WFX347TCD3F7',
 'UN6JA0KEMF0Z',
 'DT3C386HN1DW',
 '0MCDJR3HU2XR',
 '2K37ZE3ZGAFQ',
 'GGZYMZ1TTVZ5',
 'A2KC8SEN51EO',
 'IGQ9TF2RW749',
 '3Z5MVM1UNXO5',
 '1IMW1KANT2AD',
 'NJ7TT302OARG',
 '2JY2P8U3T97N',
 'S6L0ZXFB3554',
 '0EZ2BV49DXD7

In [42]:
train[train.user_id=='RR9IQRXGTB57'].action_type.unique()

array(['search for poi'], dtype=object)

In [43]:
test[test.user_id=='RR9IQRXGTB57'].action_type.unique()

array([], dtype=object)

In [ ]:
has[has].index

In [ ]:
has[~has].index

In [ ]:
len(has[~has].index)

In [ ]:
has[~has].index.isin(has[has].index)

In [ ]:
'052f864e8a1c0'

In [ ]:
train[train.session_id=='052f864e8a1c0'].action_type.unique()

TO dos:
    
    1. check why there are sessions without clickouts